In [1]:
# Initial Configuration
import findspark 
findspark.init()

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local') # we are hosting clusters locally.
spark = SparkSession(sc) # create an interface to the spark session.

In [2]:
df = spark.read.parquet('hmp.parquet')

In [3]:
from pyspark.ml.feature import VectorAssembler

vectorAssembler = VectorAssembler(inputCols=["x","y","z"],
                                  outputCol="features")

In [4]:
from pyspark.ml.clustering import KMeans

kmeans = KMeans().setK(13).setSeed(1)

In [5]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[vectorAssembler, kmeans])

In [6]:
df.createOrReplaceTempView('df')
df = spark.sql("select * from df where class in ('Brush_teeth','Climb_stairs')")

In [7]:
model = pipeline.fit(df)

In [8]:
wssse = model.stages[1].computeCost(vectorAssembler.transform(df))
print("Within Set Sum of Squared Errors = " + str(wssse))

Within Set Sum of Squared Errors = 2588490.500006362


### Practise

In [9]:
# download the data
!wget https://github.com/IBM/coursera/raw/master/coursera_ml/a2.parquet

SYSTEM_WGETRC = c:/progra~1/wget/etc/wgetrc
syswgetrc = C:\Program Files (x86)\Gow/etc/wgetrc
--2019-06-14 13:59:46--  https://github.com/IBM/coursera/raw/master/coursera_ml/a2.parquet
Resolving github.com... 13.234.176.102
Connecting to github.com|13.234.176.102|:443... connected.
OpenSSL: error:1407742E:SSL routines:SSL23_GET_SERVER_HELLO:tlsv1 alert protocol version
Unable to establish SSL connection.


In [11]:
# Read the data in PySpark Dataframe
df = spark.read.load(r'C:\Users\ashutosh.b.patel\Desktop\Data Science and Engineering\BITS\Coursera\coursera\coursera_ml\a2.parquet')

# Create a Spark SQL table
df.createOrReplaceTempView("df")
spark.sql("SELECT * from df").show()

+-----+-----------+-------------------+-------------------+-------------------+
|CLASS|   SENSORID|                  X|                  Y|                  Z|
+-----+-----------+-------------------+-------------------+-------------------+
|    0|         26| 380.66434005495194| -139.3470983812975|-247.93697521077704|
|    0|         29| 104.74324299209692| -32.27421440203938|-25.105013725863852|
|    0| 8589934658| 118.11469236129976| 45.916682927433534| -87.97203782706572|
|    0|34359738398| 246.55394030642543|-0.6122810693132044|-398.18662513951506|
|    0|17179869241|-190.32584900181487|  234.7849657520335|-206.34483804019288|
|    0|25769803830| 178.62396382387422| -47.07529438881511|  84.38310769821979|
|    0|25769803831|  85.03128805189493|-4.3024316644854546|-1.1841857567516714|
|    0|34359738411| 26.786262674736566| -46.33193951911338| 20.880756008396055|
|    0| 8589934592|-16.203752396859194| 51.080957032176954| -96.80526656416971|
|    0|25769803852|   47.2048142440404| 

Let’s check if we have balanced classes – this means that we have roughly the same number of examples for each class we want to predict. This is important for classification but also helpful for clustering

In [12]:
spark.sql("SELECT count(class), class from df group by class").show()

+------------+-----+
|count(class)|class|
+------------+-----+
|        1416|    1|
|        1626|    0|
+------------+-----+



Let's create a VectorAssembler which consumes columns X, Y and Z and produces a column “features”

In [13]:
from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler(inputCols=["X","Y","Z"],
                                  outputCol="features")

In [14]:
# Instantiate the KMeans clustering algorithm with 2 clusters
from pyspark.ml.clustering import KMeans

clust = KMeans().setK(2).setSeed(1)

In [15]:
from pyspark.ml.clustering import GaussianMixture

clust = GaussianMixture().setK(2).setSeed(47)

In [16]:
# Train the model
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[vectorAssembler, clust])
model = pipeline.fit(df)

In [17]:
prediction = model.transform(df)
prediction.show()

+-----+-----------+-------------------+-------------------+-------------------+--------------------+----------+--------------------+
|CLASS|   SENSORID|                  X|                  Y|                  Z|            features|prediction|         probability|
+-----+-----------+-------------------+-------------------+-------------------+--------------------+----------+--------------------+
|    0|         26| 380.66434005495194| -139.3470983812975|-247.93697521077704|[380.664340054951...|         1|[6.17374239553848...|
|    0|         29| 104.74324299209692| -32.27421440203938|-25.105013725863852|[104.743242992096...|         1|[1.17386801362225...|
|    0| 8589934658| 118.11469236129976| 45.916682927433534| -87.97203782706572|[118.114692361299...|         1|[2.95529882990642...|
|    0|34359738398| 246.55394030642543|-0.6122810693132044|-398.18662513951506|[246.553940306425...|         1|[4.27416512058169...|
|    0|17179869241|-190.32584900181487|  234.7849657520335|-206.34483

In [18]:
# Check the accuracy of the model
prediction.createOrReplaceTempView('prediction')
spark.sql('''
select max(correct)/max(total) as accuracy from (

    select sum(correct) as correct, count(correct) as total from (
        select case when class != prediction then 1 else 0 end as correct from prediction 
    ) 
    
    union
    
    select sum(correct) as correct, count(correct) as total from (
        select case when class = prediction then 1 else 0 end as correct from prediction 
    ) 
)
''').rdd.map(lambda row: row.accuracy).collect()[0]

0.9927679158448389

The model has perfectly clustered almost every data point into 2 cluster.